In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmaps
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import cartopy.feature as cfea

In [2]:
lat_shape = 21600
lon_shape = 43200
dtype='float32'
SUF = '30s'
area_path = f'/mnt/c/Users/tsimk/Downloads/H08_20230612/map/dat/grd_ara_/grdara.{SUF}'
area = np.fromfile(area_path, dtype=dtype).reshape(lat_shape, lon_shape)
print(np.max(area))

854796.56


In [3]:
lat_shape = 21600
lon_shape = 43200
dtype='float32'
SUF = '30s'
h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city'
pop_path = f'{h08dir}/dat/pop_tot_/GPW4ag__20100000.{SUF}'
gwp_pop = np.fromfile(pop_path, dtype=dtype).reshape(lat_shape, lon_shape)
print(np.max(gwp_pop))
gwp_pop = np.where(gwp_pop == 1e20, 0, gwp_pop)
print(np.max(gwp_pop))

1e+20
141715.3


In [4]:
def geography():
    SUF = '.30s'
    dtype = 'float32'
    #
    a = 21600
    b = 43200
    shape = (a, b)
    #
    # tokyo
    city_num = 1
    #
    # region
    one = 138
    two = 141
    three = 34
    four = 37
    #
    resolution = 120
    upperindex = (90-four)*resolution
    lowerindex = (90-three)*resolution
    leftindex = (180+one)*resolution
    rightindex = (180+two)*resolution
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (one, two, three, four)
    
    return a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num

In [5]:
def geography():
    dtype = 'float32'
    #
    a = 21600
    b = 43200
    shape = (a, b)
    #
    # Al-Madinah(Medina)
    city_num = 393
    #
    # region
    one = 37
    two = 41
    three = 23
    four = 26
    #
    resolution = 120
    upperindex = (90-four)*resolution
    lowerindex = (90-three)*resolutioion
    leftindex = (180+one)*resolutioion
    rightindex = (180+two)*resolutioion
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (one, two, three, four)
    
    return a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num

In [134]:
def geography():
    dtype = 'float32'
    #
    a = 21600
    b = 43200
    gl5shape = (a, b)
    #
    # Ar-Riyadh (Riyadh)
    city_num = 61
    #
    # region
    one = 45
    two = 49
    three = 23
    four = 26
    #
    resolution = 120
    upperindex = (90-four)*resolution
    lowerindex = (90-three)*resolution
    leftindex = (180+one)*resolution
    rightindex = (180+two)*resolution
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (one, two, three, four)
    
    return a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num

In [6]:
def geography():
    dtype = 'float32'
    #
    a = 21600
    b = 43200
    gl5shape = (a, b)
    #
    # beiging
    city_num = 10
    #
    # region
    one = 115
    two = 118
    three = 39
    four = 41
    #
    resolution = 120
    upperindex = (90-four)*resolution
    lowerindex = (90-three)*resolution
    leftindex = (180+one)*resolution
    rightindex = (180+two)*resolution
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (one, two, three, four)
    
    return a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num

In [10]:
def lonlat2xy(n0x, n0y, p0lonmin, p0lonmax, p0latmin, p0latmax, r0lon, r0lat):
    n0x = int(n0x)
    n0y = int(n0y)
    p0lonmin = float(p0lonmin)
    p0lonmax = float(p0lonmax)
    p0latmin = float(p0latmin)
    p9latmax = float(p0latmax)
    r0lon = float(r0lon)
    r0lat = float(r0lat)

    if p0lonmin <= r0lon <= p0lonmax:
        i0x = int((r0lon - p0lonmin)/(p0lonmax - p0lonmin) * float(n0x))
    else:
        i0x = 1e20

    if p0latmin <= r0lat <= p0latmax:
        i0y = int((p0latmax - r0lat)/(p0latmax - p0latmin) * float(n0y))
    else:
        i0y = 1e20

    return i0x, i0y

def xy2lonlat(n0x, n0y, p0lonmin, p0lonmax, p0latmin, p0latmax, i0x, i0y):
    if 0 <= i0x <= n0x:
        rgetlon = p0lonmin + ((p0lonmax - p0lonmin)/float(n0x)) * (float(i0x) + 0.5)
        rgetlat = p0latmax - ((p0latmax - p0latmin)/float(n0y)) * (float(i0y) + 0.5)
    else:
        rgetlon = 1e20
        rgetlat = 1e20
    
    return rgetlon, rgetlat
    
def draw_mask(mask_temp, pop_temp, city_index, loop_num, maximum_value, best_masked_pop, un_pop, best_coverage, density_ratio):
    # cmap
    pop_vmax = 1e5
    
    # get city lonlat
    a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num = geography()

    # crop
    mask_cropped = mask_temp[upperindex:lowerindex, leftindex:rightindex]
    
    # pop
    pop_cropped = pop_temp[upperindex:lowerindex, leftindex:rightindex]
    
    """
    # josui
    josui_path = f"/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/fcl_obs_/jyousui_20240109.csv"
    josui = pd.read_csv(josui_path)
    
    # regionalize
    josui_array = np.zeros((a, b))
    # 
    for i in range(len(josui['lon'])):
        lon_middle, lat_middle = lonlat2xy(43200, 21600, -180, 180, -90, 90, josui['lon'][i], josui['lat'][i])
        # missing value in csv
        if lon_middle >= b or lat_middle >= a:
            continue
        else:
            josui_array[lat_middle, lon_middle] = 1
    #        
    josui_cropped = josui_array[upperindex:lowerindex, leftindex:rightindex]
    josui_masked = np.ma.masked_where(josui_cropped==0, josui_cropped)
    """

    # extent
    projection = ccrs.PlateCarree()

    # figure
    fig = plt.figure(figsize=(10,10))
    ax = plt.subplot(projection=projection)
    ax.coastlines()
    ax.set_extent(img_extent)
    ax.gridlines(draw_labels=True)
    im = ax.imshow(pop_cropped,
                   origin='upper',extent=img_extent, transform=projection, cmap='gist_heat_r'
                   #vmin=0, vmax=pop_vmax
                  )
    #
    seethrough = np.ma.masked_where(mask_cropped == 0, mask_cropped)
    seethrough[-1, -1] = 0
    im2 = ax.imshow(seethrough,
                    origin='upper',extent=img_extent, transform=projection, cmap='gray_r'
                   )

    # josui plot
    # 
    #josui_cmap = plt.cm.colors.ListedColormap(['white', 'blue'])
    #im3 = ax.imshow(josui_masked, origin='upper', cmap=josui_cmap, vmin=0, vmax=1, extent=img_extent,  transform=projection, zorder=3)
    #im4 = ax.scatter(josui['lon'], josui['lat'], color='blue', s=5, zorder=4)
    
    # save figure
    gif_path = f'/mnt/c/Users/tsimk/Downloads/research/H08/tokyomodel/citymask_30s_gifs/{loop_num:08}.jpg'
    plt.title(f"population density: {maximum_value} [/km2]\ndensity ratio: {density_ratio}[%]\ncoverage: {int(best_masked_pop)}/{int(un_pop)}[person]\ncoverage rate: {np.round(best_coverage*100,2)}[%]")
    plt.savefig(gif_path)

    #fig.colorbar(im, ax=ax, orientation='horizontal')
    #plt.show()

In [11]:
def explore_citymask(index):
    """
    downtown mask will stop at 12% and 400masks in tokyo
    In high resolution, is downtown algorithm no longer available??
    """

    #-----------------------------------------------
    # Initialization
    #-----------------------------------------------

    # suffix
    SUF = '30s'
    
    # pop data
    POP='gpw4_30s'
    
    # map data
    MAP='CAMA'

    # search radius (1grid in 0.5degree)
    circle = 30

    # EN.1: lower limitation of population density
    lowlim = 1000
    
    # EN.2: initial grid threshold
    threshold = 1000

    # EN.3: grid sum
    grdlim = 1000

    # EN.3: downtown rate
    downtown_rate = 1.5
    
    # EN.4: lowest rate
    lowrat = 0.0

    # shape
    lat_shape = 21600
    lon_shape = 43200

    # date type
    dtype= 'float32'

    # h08 directory
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city'

    # initialize variables
    best_coverage = float('inf')
    best_mask = None
    best_masked_pop = None

    #-----------------------------------------------
    # load true data (UN city list) unit=[1000person]
    #-----------------------------------------------

    # true population and city name
    un_pop_list = []
    name_list = []

    # load data
    city_path = f'{h08dir}/dat/cty_lst_/gpw4/WUP2018_300k_2010.txt'
    for l in open(city_path).readlines():
        data = l[:].split('\t')
        data = [item.strip() for item in data]
        un_pop_list.append(float(data[3]))
        name_list.append(data[4])

    # get true UN city population
    un_pop = un_pop_list[index-1]*1000

    # get city name
    city_name = name_list[index-1]

    #-----------------------------------------------
    #  Get area(m2)
    #-----------------------------------------------

    area_path = f'/mnt/c/Users/tsimk/Downloads/H08_20230612/map/dat/grd_ara_/grdara.{SUF}'
    area = np.fromfile(area_path, dtype=dtype).reshape(lat_shape, lon_shape)

    #-----------------------------------------------
    # load gwp population data
    #-----------------------------------------------

    # population data(GWP4 2000)
    pop_path = f'{h08dir}/dat/pop_tot_/GPW4ag__20100000.{SUF}'
    gwp_pop = np.fromfile(pop_path, dtype=dtype).reshape(lat_shape, lon_shape)
    gwp_pop = np.where(gwp_pop == 1e20, 0, gwp_pop)

    # population density (person/km2)
    gwp_pop_density = (gwp_pop / (area / 10**6))

    #-----------------------------------------------
    # load city_center coordinate
    #-----------------------------------------------

    center_path = f'{h08dir}/dat/cty_cnt_/{POP}/cityclrd0000.{SUF}'
    location = np.fromfile(center_path, dtype=dtype).reshape(lat_shape,lon_shape)
    org_y = np.where(location==index)[0]
    org_x = np.where(location==index)[1]
    org_y = org_y[0]
    org_x = org_x[0]
    
    #-----------------------------------------------
    # check city center
    #-----------------------------------------------

    # original city center
    org_cnt = gwp_pop_density[org_y, org_x]

    # number of replacement
    print(f"cityindex {index}")
    org_lon, org_lat = xy2lonlat(lon_shape, lat_shape, -180, 180, -90, 90, org_x, org_y)
    print(f'original center [x, y] = [{org_lon, org_lat}]')
    print(f'org_cnt: {org_cnt}')
    
    for a_cnt in range(org_y-circle, org_y+circle+1):
        for b_cnt in range(org_x-circle, org_x+circle+1):
            candidate = gwp_pop_density[a_cnt, b_cnt]
            if candidate >= org_cnt:
                org_cnt = candidate
                rpl_y = a_cnt
                rpl_x = b_cnt
                
    new_lon, new_lat = xy2lonlat(lon_shape, lat_shape, -180, 180, -90, 90, rpl_x, rpl_y)
    print(f'replaced center [x, y] = [{new_lon, new_lat}]')
    print(f"rpl_cnt: {gwp_pop_density[rpl_y, rpl_x]}")

    #-----------------------------------------------
    #  Initialization of mask array
    #-----------------------------------------------

    # mask array for saving
    mask = np.zeros((lat_shape,lon_shape), dtype=dtype)
    mask[rpl_y, rpl_x] = 1

    #-----------------------------------------------
    #  Explore start
    #-----------------------------------------------

    # stop flag
    new_mask_added = True
    coverage_flag = True

    # city center
    best_mask = mask
    grid_num = np.sum(best_mask)
    best_masked_pop = np.sum(gwp_pop*mask)
    best_coverage = float(best_masked_pop / un_pop)

    # monitor density ration
    init_masked_dns = np.sum(gwp_pop_density[rpl_y, rpl_x])
    previous_density = np.sum(gwp_pop_density[rpl_y, rpl_x])

    # initial grid threshold
    if gwp_pop_density[rpl_y, rpl_x] <= threshold:
        print("/// stop ///")
        print("/// stop ///")
        print(f"initial density {gwp_pop_density[rpl_y, rpl_x]} less than threshold {threshold}")
        print("/// stop ///")
        print("/// stop ///")
        new_mask_added = False
        coverage_flag = False
        loop_num = 1
        next_density = 0
        density_ratio = 0
        draw_mask(best_mask, gwp_pop_density, index, loop_num, np.round(next_density), best_masked_pop, un_pop, best_coverage, density_ratio)

    # loop start
    loop_num = 0
    while new_mask_added:
        
        ### make search list
        search_lst = []
        new_mask_added = False
        indices = np.where(mask == 1)
        
        for ind in range(len(indices[0])):
            y_index = indices[0][ind]
            x_index = indices[1][ind]
            for dy, dx in [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, 1)]:
                i = y_index + dy
                j = x_index + dx
                # not explored yet
                if mask[i, j] == 0:
                    # within grid range
                    if 0 <= i < lat_shape and 0<= j < lon_shape:
                        search_lst.append([gwp_pop_density[i, j], i, j])

        ### add searched grid
        # empty check
        if not search_lst:

            print('search_lst is empty')
            new_mask_added = False
            coverage_flag = False
            
        # get largest grid
        else:
            
            sorted_search = sorted(search_lst, key=lambda x: x[0], reverse=True)
            largest = sorted_search[0]
            next_density = gwp_pop_density[largest[1], largest[2]]
            density_ratio = int((next_density/init_masked_dns)*100)
            
            # if largest grid value is too small, stop exploring
            if next_density <= lowlim:
                print("/// stop ///")
                print("/// stop ///")
                print(f"largest density {gwp_pop_density[largest[1], largest[2]]} smaller than lowlim {lowlim}")
                print([gwp_pop_density[t[1], t[2]]for t in sorted_search])
                print("/// stop ///")
                print("/// stop ///")
                new_mask_added = False
                coverage_flag = False
                
            elif next_density > previous_density and best_coverage > downtown_rate and grid_num >= grdlim:
                print("/// stop ///")
                print("/// stop ///")
                print(f"latest density {next_density} bigger than previous density {previous_density}")
                print("/// stop ///")
                print("/// stop ///")
                new_mask_added = False
                coverage_flag = False
                
            elif density_ratio < lowrat:
                print("/// stop ///")
                print("/// stop ///")
                print(f"latest density {next_density} less than 1/{lowrat} of initial density {init_masked_dns}")
                print("/// stop ///")
                print("/// stop ///")
                new_mask_added = False
                coverage_flag = False
                
        # stop flag
        if coverage_flag is True:
                new_mask_added = True

                # evaluate coverage
                gwp_masked_pop = np.sum(mask * gwp_pop)
                coverage = float(gwp_masked_pop / un_pop)

                # stop exploring
                if coverage >= 1.0:
                    new_mask_added = False
                    coverage_flag = False

                # judge
                judge_value = abs(1 - coverage)
                best_value = abs(1 - best_coverage)
                
                # update
                if judge_value < best_value:
                    best_coverage = coverage
                    best_mask = mask
                    best_masked_pop = gwp_masked_pop
                    grid_num = np.sum(best_mask)
                    previous_density = gwp_pop_density[largest[1], largest[2]]
        
        # search_lst is colored
        for cand in search_lst:
            mask[cand[1], cand[2]] = 0.1
        # for drawing
        mask[largest[1], largest[2]] = 0.3
        
        # draw
        # cand = 0.3
        # largest = 0.6
        # citymask = 1
        density_ratio = int((next_density/init_masked_dns)*100)
        draw_mask(best_mask, gwp_pop_density, index, loop_num, np.round(next_density), best_masked_pop, un_pop, best_coverage, density_ratio)
        #new_mask_added = False # this should be removed in the future
        #coverage_flag = False # this should be removed in the future
        
        # reset
        mask[mask == 0.1] = 0
        mask[mask == 0.3] = 1
        # loop count
        loop_num += 1

        #-----------------------------------------------
        # Output result
        #-----------------------------------------------

        print('#########################################')
        print(
              f"explored_pop {best_masked_pop}\n" \
              f"true_pop {un_pop}\n" \
              f"coverage {best_coverage}\n" \
              f"city_mask {grid_num}\n" \
              f"{city_name}"
              )
        print('#########################################')

In [12]:
def check_overlap(prelim_mask, new_mask):
    if np.any(np.logical_and(prelim_mask > 0, new_mask > 0)):
        print(f"overlap detected")
    else:
        print(f"no overlap")
        
prelim_mask = np.zeros((5, 10))
new_mask = np.ones((5, 10))
check_overlap(prelim_mask, new_mask)

prelim_mask[0,0] = 1
check_overlap(prelim_mask, new_mask)

no overlap
overlap detected


In [13]:
import numpy as np

mask = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

print(mask.shape)

indices = np.where(mask == 1)

for i in range(len(indices[0])):
    y_index, x_index = indices[0][i], indices[1][i]
    print(f"Index {i + 1}: y={y_index}, x={x_index}")

(3, 4)
Index 1: y=0, x=0
Index 2: y=1, x=2
Index 3: y=2, x=3


In [ ]:
# execute
a, b, upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent, city_num = geography()
explore_citymask(city_num)

cityindex 1
original center [x, y] = [(139.69583333333333, 35.6875)]
org_cnt: 16290.229661929157
replaced center [x, y] = [(139.71249999999998, 35.72083333333333)]
rpl_cnt: 21744.161202840052
#########################################
explored_pop 15141.76953125
true_pop 36859626.0
coverage 0.00041079552818170214
city_mask 1.0
Tokyo
#########################################
#########################################
explored_pop 30281.9609375
true_pop 36859626.0
coverage 0.0008215482419029428
city_mask 2.0
Tokyo
#########################################
#########################################
explored_pop 45422.1484375
true_pop 36859626.0
coverage 0.001232300849647796
city_mask 3.0
Tokyo
#########################################
#########################################
explored_pop 60562.33984375
true_pop 36859626.0
coverage 0.001643053563369037
city_mask 4.0
Tokyo
#########################################
#########################################
explored_pop 75702.53125
true_pop 368

/tmp/ipykernel_474/3431094078.py:70: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10,10))


#########################################
explored_pop 308380.28125
true_pop 36859626.0
coverage 0.008366343197567985
city_mask 21.0
Tokyo
#########################################
#########################################
explored_pop 322417.90625
true_pop 36859626.0
coverage 0.00874718333414452
city_mask 22.0
Tokyo
#########################################
#########################################
explored_pop 336454.0625
true_pop 36859626.0
coverage 0.009127983623599437
city_mask 23.0
Tokyo
#########################################
#########################################
explored_pop 350488.75
true_pop 36859626.0
coverage 0.009508744065932736
city_mask 24.0
Tokyo
#########################################
#########################################
explored_pop 364523.4375
true_pop 36859626.0
coverage 0.009889504508266036
city_mask 25.0
Tokyo
#########################################
#########################################
explored_pop 378558.09375
true_pop 36859626.0
coverage 0.01